In [2]:
from tree_sitter import Language, Parser

In [ ]:
Language.build_library(
    # Store the library in the `build` directory
    "build/my-languages.so",
    # Include one or more languages
    ["vendor/tree-sitter-java", "vendor/tree-sitter-python"],
)

In [ ]:
PY_LANGUAGE = Language("build/my-languages.so", "python")

In [ ]:
parser = Parser()
parser.set_language(PY_LANGUAGE)

In [ ]:
tree = parser.parse(
    bytes(
        """
def foo():
    if bar:
        baz()
""",
        "utf8",
    )
)

In [ ]:
cursor = tree.walk()

assert cursor.node.type == "module"

assert cursor.goto_first_child()
assert cursor.node.type == "function_definition"

assert cursor.goto_first_child()
assert cursor.node.type == "def"

# Returns `False` because the `def` node has no children
assert not cursor.goto_first_child()

assert cursor.goto_next_sibling()
assert cursor.node.type == "identifier"

assert cursor.goto_next_sibling()
assert cursor.node.type == "parameters"

assert cursor.goto_parent()
assert cursor.node.type == "function_definition"

In [3]:
JAVA_LANGUAGE = Language("build/my-languages.so", "java")

In [4]:
parser = Parser()
parser.set_language(JAVA_LANGUAGE)

In [30]:
code = """
@ApiOperation(value = "Get persistent RPC request", notes = "Get information about the status of the RPC call." + TENANT_OR_CUSTOMER_AUTHORITY_PARAGRAPH)
@PreAuthorize("hasAnyAuthority('TENANT_ADMIN', 'CUSTOMER_USER')")
@RequestMapping(value = "/persistent/{rpcId}", method = RequestMethod.GET)
@ResponseBody
public Rpc getPersistedRpc(
        @ApiParam(value = RPC_ID_PARAM_DESCRIPTION, required = true)
        @PathVariable(RPC_ID) String strRpc) throws ThingsboardException {
    checkParameter("RpcId", strRpc);
    RpcId rpcId = new RpcId(UUID.fromString(strRpc));
    return checkRpcId(rpcId, Operation.READ);
}
"""

In [31]:
tree = parser.parse(
    bytes(
        code,
        "utf8",
    )
)

In [32]:
s_expression = tree.root_node.sexp()
print(s_expression)

(program (method_declaration (modifiers (annotation name: (identifier) arguments: (annotation_argument_list (element_value_pair key: (identifier) value: (string_literal (string_fragment))) (element_value_pair key: (identifier) value: (binary_expression left: (string_literal (string_fragment)) right: (identifier))))) (annotation name: (identifier) arguments: (annotation_argument_list (string_literal (string_fragment)))) (annotation name: (identifier) arguments: (annotation_argument_list (element_value_pair key: (identifier) value: (string_literal (string_fragment))) (element_value_pair key: (identifier) value: (field_access object: (identifier) field: (identifier))))) (marker_annotation name: (identifier))) type: (type_identifier) name: (identifier) parameters: (formal_parameters (formal_parameter (modifiers (annotation name: (identifier) arguments: (annotation_argument_list (element_value_pair key: (identifier) value: (identifier)) (element_value_pair key: (identifier) value: (true))))

In [33]:
query = JAVA_LANGUAGE.query(
"""
(method_declaration
    (modifiers
        (annotation
            "@" name: (identifier) @annotation_name)))
"""
)

captures = query.captures(tree.root_node)

In [29]:
for row in captures:
    for element in row:
        print(element, end=' ')
    print()

print(captures[0][0].start_point[0])
print(captures[0][0].start_point[1])
print(captures[0][0].end_point[0])
print(captures[0][0].end_point[1])

<Node type=identifier, start_point=(1, 1), end_point=(1, 13)> annotation_name 
<Node type=identifier, start_point=(2, 1), end_point=(2, 13)> annotation_name 
<Node type=identifier, start_point=(3, 1), end_point=(3, 15)> annotation_name 
1
1
1
13


In [43]:
code_array = code.strip().split("\n")
print(code_array)
print(code_array[captures[0][0].start_point[0] - 1])
print(code_array[captures[0][0].start_point[0] - 1][captures[0][0].start_point[1]: captures[0][0].end_point[1]])

['@ApiOperation(value = "Get persistent RPC request", notes = "Get information about the status of the RPC call." + TENANT_OR_CUSTOMER_AUTHORITY_PARAGRAPH)', '@PreAuthorize("hasAnyAuthority(\'TENANT_ADMIN\', \'CUSTOMER_USER\')")', '@RequestMapping(value = "/persistent/{rpcId}", method = RequestMethod.GET)', '@ResponseBody', 'public Rpc getPersistedRpc(', '        @ApiParam(value = RPC_ID_PARAM_DESCRIPTION, required = true)', '        @PathVariable(RPC_ID) String strRpc) throws ThingsboardException {', '    checkParameter("RpcId", strRpc);', '    RpcId rpcId = new RpcId(UUID.fromString(strRpc));', '    return checkRpcId(rpcId, Operation.READ);', '}']
@ApiOperation(value = "Get persistent RPC request", notes = "Get information about the status of the RPC call." + TENANT_OR_CUSTOMER_AUTHORITY_PARAGRAPH)
ApiOperation
